In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd  # For optional Pandas DataFrame handling

def scrape_website(url, data_type):
    """
    Scrapes a website for specified data and saves it to a CSV file.

    Args:
        url: The URL of the website to scrape.
        data_type: The type of data to scrape (e.g., "headlines", "products", "jobs").
    """

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
        soup = BeautifulSoup(response.content, "html.parser")

        all_data = []

        if data_type == "headlines":
            headlines = scrape_headlines(soup)
            all_data.extend(headlines) # handles pagination if available

        elif data_type == "products":
            products = scrape_products(soup)
            all_data.extend(products)
            # Add pagination handling if needed for product pages

        elif data_type == "jobs":
            jobs = scrape_jobs(soup)
            all_data.extend(jobs)
            # Add pagination handling if needed for job listings

        else:
            print("Invalid data type specified.")
            return

        if all_data: # Check if any data was extracted
            save_to_csv(all_data, f"{data_type}_data.csv")
            # Or use pandas for more structured data handling:
            # df = pd.DataFrame(all_data)
            # df.to_csv(f"{data_type}_data.csv", index=False)  # Save to CSV using pandas
            print(f"Data scraped and saved to {data_type}_data.csv")
        else:
            print("No data found matching the specified criteria.")

    except requests.exceptions.RequestException as e:
        print(f"Error during request: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")


def scrape_headlines(soup):
    headlines = []
    for h2_tag in soup.find_all("h2"):  # Example: Scrape h2 tags for headlines
        headline_text = h2_tag.text.strip()
        if headline_text: # Check for empty strings
          headlines.append({"headline": headline_text})  # Store in dictionary for CSV
    return headlines


def scrape_products(soup):
    products = []
    # Implement your logic to extract product details (name, price, etc.)
    # Example (replace with your website's specific structure):
    for product_item in soup.find_all("div", class_="product-item"):  # Example class
        name = product_item.find("a", class_="product-name").text.strip() if product_item.find("a", class_="product-name") else "N/A"
        price = product_item.find("span", class_="product-price").text.strip() if product_item.find("span", class_="product-price") else "N/A"
        products.append({"name": name, "price": price})
    return products


def scrape_jobs(soup):
    jobs = []
    # Implement your logic to extract job details (title, company, location, etc.)
    # Example (replace with your website's specific structure):
    for job_listing in soup.find_all("li", class_="job-listing"):  # Example class
        title = job_listing.find("h3").text.strip() if job_listing.find("h3") else "N/A"
        company = job_listing.find("span", class_="company").text.strip() if job_listing.find("span", class_="company") else "N/A"
        jobs.append({"title": title, "company": company})
    return jobs


def save_to_csv(data, filename):
    if data:
        keys = data[0].keys()  # Get the keys (column names) from the first dictionary
        with open(filename, "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=keys)
            writer.writeheader()
            writer.writerows(data)



# Example usage:
url = input("Enter the URL of the website: ")
data_type = input("Enter the type of data to scrape (headlines, products, jobs): ").lower()

scrape_website(url, data_type)